# About this Page
See [**slack #ts-logging**](https://rubin-obs.slack.com/archives/C07QJMQKJE6)
channel for discussion about this page.  There you can report problems, questions, and requests for changes.

## What is new in this app?(newest change at top of list)
- Do not show Time Loss info unless it is greater than zero
- Do not show software or hardware components (they are rarely used anyhow)
- <font style="background-color:#ffff99">Embedded comments and questions to reader (Yellow background)</font>
- This page tries to work around existing problems with APIs used for source data. (There is a ticket to fix the APIs)
- <font style="background-color:red; color:#90EE90">Errors from APIs are displayed in this page (Red background).</font>


In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "TODAY"  # TODO Change to 'YESTERDAY' and 'TODAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

verbose = "false"  # TODO change to false before push, else true

In [2]:
# IMPORT everything
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [3]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", 
    ut.Server.usdf) # TODO try with "usdf" before push (also "summit")

In [4]:
date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

verbose = (verbose == "true") 


In [5]:
# Read records from (almost) all sources
timer = ut.Timer()
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=verbose,                
    limit=5000,
    exclude_instruments = []      # TODO change to empty list before push
)
allrep = AllReports(allsrc=allsrc)

----------

In [6]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")
md(f"Most recently run on: **{dt.datetime.now().replace(microsecond=0).isoformat(sep=' ')}**")

# Showing data for 2024-11-20 to 2024-11-21 for LSSTComCamSim, LATISS, LSSTComCam, LSSTCam

Most recently run on: **2024-11-20 22:12:27**

# Table of Contents
* [Night Report](#Night-Report-BETA)
    - AuxTel
    - Simonyi
* [Almanac](#Almanac-BETA)
* [Summary plots of whole night](#Summary-plots-of-whole-night-DRAFT)
* [Time Accounting](#Time-Accounting-DRAFT)
* [Jira Tickets](#Jira-Tickets-BETA)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed-DRAFT)
    - AuxTel
    - Simonyi
* [Data Log](#Data-Log-BETA)
* [Narrative Log](#Narrative-Log-BETA)
* [Developer Only Section](#Developer-Only-Section-REMOVE)

## Night Report <font style="background-color:green; color:white; font-size:20px">BETA</font> 
<font style="background-color:#ffff99">
Word wrap instead of chopping words at end of line?
</font>

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

<font style="color:lightblue;background-color:None;">No nightreport records found 2024-11-20 to 2024-11-21. </font>

Used [API Data](https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241120&max_day_obs=20241121)

## Almanac <font style="background-color:green; color:white; font-size:20px">BETA</font> 
<font style="background-color:#ffff99">
Add column for Chile timezone?
</font>

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-11-20**

Sun Set,2024-11-20 23:17:44,
Evening Civil Twilight,2024-11-20 23:48:28,(-6 degrees)
Evening Nautical Twilight,2024-11-21 00:20:27,(-12 degrees)
Evening Astronomical Twilight,2024-11-21 00:54:12,(-18 degrees)
Moon Rise,2024-11-21 04:26:54,
Solar Midnight,2024-11-21 04:28:45,
Morning Astronomical Twilight,2024-11-21 08:03:18,(-18 degrees)
Morning Nautical Twilight,2024-11-21 08:37:03,(-12 degrees)
Morning Civil Twilight,2024-11-21 09:09:02,(-6 degrees)
Sun Rise,2024-11-21 09:39:47,
Moon Set,2024-11-21 14:51:23,


## Summary plots of whole night <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

(content not yet defined in storyboard)

In [9]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

## DDV <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

In [10]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

Access DDV part of RubinTV: https://usdf-rsp-dev.slac.stanford.edu/rubintv/ddv/index.html

## Time Accounting <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
TODO:
- Time-loss for Fault and Weather
- SlewTime from TMAEvent (or similar)

SlewTime (and probably others) from EFD are only EXPECTED (topic="lsst.sal.Scheduler.logevent_target"), not ACTUAL.
To get ACTUAL time, we need to use something like 
[**TMAEvent**](https://github.com/lsst-sitcom/summit_utils/blob/f901ca4943c082500a1f357eb070a4971925c476/python/lsst/summit/utils/tmaUtils.py#L1195) 
to combine multiple real events into a synthetic event.  See DM-46102.


In [11]:
# Exposure Tally
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

,LSSTComCam
Total Night,7:09:05
Total Exposure,0:00:49
Slew time(1),0:00:00
Readout time(2),0:00:45
Time loss to fault,0
Time loss to weather,0
Idle time,7:07:30
Number of exposures,19
Mean readout time,0:00:02
Number of slews(1),0


- (1) *Expected* slew time as per Schedular
- (2) There is no practical way to get detector read-out time.  A value of 2.3 seconds was inferred from experiments.

## Jira Tickets <font style="background-color:green; color:white; font-size:20px">BETA</font> 
- Observing Operations - OBS
- TODO: better tickets using Jira API

In [12]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

- [OBS-693](https://rubinobs.atlassian.net/browse/OBS-693)

- [OBS-695](https://rubinobs.atlassian.net/browse/OBS-695)

- [OBS-694](https://rubinobs.atlassian.net/browse/OBS-694)

- [OBS-696](https://rubinobs.atlassian.net/browse/OBS-696)

In [13]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

No tickets found using: [API Data](https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241120&max_day_obs=20241121) in `confluence_url`.

## BLOCKS Observed <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
- TODO: BLOCKS observed

## Data Log <font style="background-color:green; color:white; font-size:20px">BETA</font>

In [14]:
# Exposure Report
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    df_dict = allsrc.fields_count_exposure(instrum)
    for field_name,df in df_dict.items():
        if df.empty:
            continue
        print(field_name.title().replace('_',' '))
        display(HTML(df.style.hide().to_html(escape=False)))

**LSSTComCamSim: 0 total exposures**

**LATISS: 0 total exposures**

**LSSTComCam: 19 total exposures**

Observation Type


Detail,good,questionable,junk,unknown
acq,0,0,0,9
bias,0,0,0,10


Observation Reason


Detail,good,questionable,junk,unknown
comcam-usdf-checkout,0,0,0,10
tmacheckout,0,0,0,9


Science Program


Detail,good,questionable,junk,unknown
BLOCK-T246,0,0,0,10
BLOCK-T250,0,0,0,9


**LSSTCam: 0 total exposures**

## Narrative Log <font style="background-color:green; color:white; font-size:20px">BETA</font> 
<font style="background-color:#ffff99">
- Truncate text messages to N chars (ending with "..." as link if truncated, link goes to page with full message record including text and all attributes)
- Detect some kinds of "blocks" and render in smaller text (+ other format changes)  
</font>

In [15]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

- **2024-11-20 10:37**   **MainTel** 

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>TMA is parked. M2 recovered. Finishing the remainder of the shut down procedures.</pre>




M2 went into fault while trying to park the TMA with elevation parking settings. We had just tried to run BLOCK-284 with no success. 


M2 is Reporting:


2024/11/20 09:46:40 TAI
Inclinometer Difference Error 


2024/11/20 09:48:08 TAI


Error when doing the basic cleanup and power off the motor.
<pre style='color: black; background-color: #FFDDDD; font-size: 0.875em; margin-left: 20px; '>Traceback (most recent call last):
  File &quot;/opt/lsst/software/stack/miniconda/lib/python3.11/site-packages/lsst/ts/m2/csc.py&quot;, line 1277, in _basic_cleanup_and_power_off_motor
    await self._execute_command(
  File &quot;/opt/lsst/software/stack/miniconda/lib/python3.11/site-packages/lsst/ts/m2/csc.py&quot;, line 2052, in _execute_command
    await command(*args, timeout=timeout, **kwargs)  # type: ignore[operator]
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File &quot;/opt/lsst/software/stack/miniconda/lib/python3.11/site-packages/lsst/ts/m2com/controller.py&quot;, line 952, in set_closed_loop_control_mode
    raise RuntimeError(
RuntimeError: Closed-loop control mode is &amp;lt;ClosedLoopControlMode.OpenLoop: 3&amp;gt; instead of &amp;lt;ClosedLoopControlMode.TelemetryOnly: 2&amp;gt; in timeout.
d
2024/11/20 09:46:40 TAI


Fault! errorCode=1, errorReport=&quot;Controller&#x27;s state is Fault.&quot;


Elevation angle as measured by M2: 70.71. 
TMA elevation position: 70.88. 


Is this why there is an inclination error? 


M2 error codes


11/20/2024 06:23:23
11/20/2024 06:23:31
11/20/2024 06:46:03
11/20/2024 06:46:03
11/20/2024 06:47:32
11/20/2024 06:50:14
6053.00
cRIO Communication Error
0.00
1.00
Controller&#x27;s state is Fault.
6060.00
Inclinometer Difference Error
</pre>


- Link: [OBS-696](https://rubinobs.atlassian.net/browse/OBS-696)

- **2024-11-20 10:02**   **MainTel** 

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>M2 error codes


11/20/2024 06:23:23
11/20/2024 06:23:31
11/20/2024 06:46:03
11/20/2024 06:46:03
11/20/2024 06:47:32
11/20/2024 06:50:14
6053.00
cRIO Communication Error
0.00
1.00
Controller's state is Fault.
6060.00
Inclinometer Difference Error</pre>


- **2024-11-20 10:01**   **MainTel** 

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Elevation angle as measured by M2: 70.71. 


TMA elevation position: 70.88. 


Is this why there is an inclination error?</pre>


...

- **2024-11-20 16:43**   **MainTel** 

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>We are back from lunch and we will resume warm-up hexapods. This time, we will run M2Hex.</pre>


- **2024-11-20 15:58**   **MainTel** 

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Freddy reports that the mechanical team is working on cleaning the TMA oil. This is transparent for telescope operations and does not block us from moving. I am trying to coordinate TMA handover for today.</pre>


- **2024-11-20 15:47**

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Warming up camera hexapod</pre>


- **2024-11-20 15:36**   **MainTel** 

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Meanwhile, we are powering up hexapods. MTHexapod:2 was ENABLED. When trying to transition to DISABLED, it went to a fault. We are now checking the cabinets to see what is their current situation. 


When checking the hexapod cabinets, we noticed that they were all off. This means that it was only the hexapod summary state that, for whatever reason, was being displayed as ENABLED. This is less concerning, since the hexapod was not energized. We are powering them up now and we will continue with start ups and warm up.</pre>


- **2024-11-20 15:32**   **MainTel** 

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Bruno and Kate already in the control room. We reviewed the HVAC set point, which is now set to 8ºC. The temperature near the sunset yesterday was 12ºC. However, the forecast temperature near sunset for today is predicted to be 10ºC. Let's leave the set point to 8ºC for now.</pre>


-----------
------------

# Developer Only Section <font style="background-color:red; olor:black; font-size:20px">REMOVE</font>
Contains stuff only expected to be useful to developers.

This may also contain sections that have moved out of the user section because they are no longer defined in the Storyboard.

## Overview 


In [16]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
- {allsrc.nig_src.timeout=}
""")

## Project-Wide Night(s) Report 
- Run on logs and databases from **https://usdf-rsp-dev.slac.stanford.edu/**
- Report **1 observing night(s)** with the last reported night starting on **2024-11-20 12:00:00**.
- min_dayobs='2024-11-20', max_dayobs='2024-11-21'
- Using ***Prototype* Logging and Reporting** Version: **52.dev0+g783922c.d20241115**
- have_consdb = False
- Imported lsst.ts.logging_and_reporting from local packages.
- allsrc.nig_src.timeout=(5.05, 20.0)


## Data Status
This may be useful for diagnosing reports that give unexpected results.

In [17]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(
    index=False, 
    render_links=True,
    escape=False)
display(HTML(table))

Endpoint,Records,URL
nightreport/reports,0,https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241120&max_day_obs=20241121
exposurelog/instruments,3,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/instruments
exposurelog/exposures.LSSTComCamSim,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241120&max_day_obs=20241121
exposurelog/exposures.LATISS,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LATISS&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241120&max_day_obs=20241121
exposurelog/exposures.LSSTComCam,19,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241120&max_day_obs=20241121
exposurelog/exposures.LSSTCam,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241120&max_day_obs=20241121
exposurelog/messages,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=2500&min_day_obs=20241120&max_day_obs=20241121
narrativelog/messages,93,https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-20T00%3A00%3A00&max_date_added=2024-11-21T00%3A00%3A00
efd/targets,0,NA


## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [18]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

### Overview for Service: `nightreport` [0]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241120&max_day_obs=20241121

None
- Got 0 records.  


### Overview for Service: `exposurelog` [0]

...

None
- Got 0 records.  


### Overview for Service: `narrativelog` [93]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-20T00%3A00%3A00&max_date_added=2024-11-21T00%3A00%3A00

None
- Got 93 records.  


In [19]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

In [20]:
# Time Log
#! allrep.exp_rep.time_log_as_markdown()

## Finale

In [21]:
print(f'Elapsed time (excluding code import): {timer.toc:.1f} seconds')
print(f"Finished {str(dt.datetime.now().replace(microsecond=0))} UTC")

Elapsed time (excluding code import): 5.9 seconds
Finished 2024-11-20 22:12:28 UTC
